In [1]:
import numpy as np
import os
from skimage.feature import hog
import cv2
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
import pandas as pd
from sklearn.svm import SVC

In [2]:
train_dir = r'ocr_data/train/'
labels_list = [i for i in os.listdir(train_dir)]
pathimg = [os.listdir(train_dir + i) for i in labels_list]

In [3]:
# hog for each image and store it in a list with its corresponding label
features = []
labels = []
for i,j in enumerate(zip(pathimg,labels_list)):
  imgs,label = j
  for img in imgs:
      img = cv2.imread(train_dir+label+'/'+img)
      img_res=cv2.resize(img,(64,128),interpolation=cv2.INTER_AREA)
      img_gray= cv2.cvtColor(img_res,cv2.COLOR_BGR2GRAY)
      hog_img= hog(img_gray,orientations=9,pixels_per_cell=(8,8), cells_per_block=(1, 1))
      features.append(hog_img)
      labels.append(label)

In [4]:
df = pd.DataFrame(np.array(features))
df['target'] = labels

#Training

In [5]:
x = np.array(df.iloc[:,:-1])
y = np.array(df['target'])

In [6]:
model = LogisticRegression()
lreg=model.fit(x, y)

C:\Users\ashuk\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


#Testing

In [8]:
test_dir = r'ocr_data/test/'

image_files = [f for f in os.listdir(test_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]

for image_file in image_files:
    image_path = os.path.join(test_dir, image_file)
    im = cv2.imread(image_path)
    im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    im_gray = cv2.resize(im_gray, (400,240))
    # Threshold 
    ret, im_th = cv2.threshold(im_gray, 120, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

    # contours 
    ctrs,hier = cv2.findContours(im_th, cv2.RETR_TREE,
                                 cv2.CHAIN_APPROX_SIMPLE)

    boxs=[cv2.boundingRect(c) for c in ctrs]
    sorted_boxs = sorted(boxs,key=lambda b:b[0])

    letters=[]
    for num,i_boxs in enumerate(sorted_boxs):
        [x,y,w,h]=i_boxs
        if h>100 and w < 100:
            r=im_gray[y:y+h,x:x+w]
            r = cv2.resize(r, (64, 128), interpolation=cv2.INTER_AREA)
            hog_fd = hog(r, orientations=9, pixels_per_cell=(8, 8),
                             cells_per_block=(1, 1))
            prediction = lreg.predict(np.array([hog_fd]))
            letters.append(str(prediction[0]))

    print(''.join(letters))



DET0UR
EAS7
N0FARKUNG
UEST
STMP
CST0BJ
ST0PJ
0NEUAY
